In [4]:
from datasets import load_dataset
import pandas as pd

# HuggingFace Hubからデータセットを取得
dataset = load_dataset("kunishou/oasst1-89k-ja", split='train')

# データセットをpandas DataFrameに変換
df = dataset.to_pandas()

# DataFrameをJSONファイルとして保存（日本語を正しく扱うために ensure_ascii=False を設定）
df.to_json("./data/oasst1-89k-ja.jsonl", force_ascii=False, orient="records", lines=True)

## oasstデータセットの取得

 - [kunishou/oasst1-89k-ja](https://huggingface.co/datasets/kunishou/oasst1-89k-ja)
 - oasstデータセットはChatデータなので、１ターンでは成立しないデータが混じっている。そのため、利用は非推奨。

In [5]:
from datasets import load_dataset
import pandas as pd
import os
import json


# oasst1のオリジナルデータのロード
ds = load_dataset("OpenAssistant/oasst1")
train = ds["train"].to_pandas()
val = ds["validation"].to_pandas()

df_origin = pd.concat([train, val], axis=0).reset_index(drop=True)

# oasst1日本語翻訳データの読み込み
df_ja = load_dataset("kunishou/oasst1-89k-ja", split='train').to_pandas()

# oasst1のオリジナルデータと日本語翻訳データのマージ
df = pd.merge(df_origin, df_ja[["message_id", "text_ja"]], on="message_id", how="left").copy()
df["text"] = df["text_ja"]

df_assistant = df[(df.role == "assistant")].copy()
df_prompter = df[(df.role == "prompter")].copy()
df_prompter = df_prompter.set_index("message_id")
df_assistant["output"] = df_assistant["text"].values

inputs = []
parent_ids = []
for _, row in df_assistant.iterrows():
    input = df_prompter.loc[row.parent_id]
    inputs.append(input.text)
    parent_ids.append(input.parent_id)

df_assistant["instruction"] = inputs
df_assistant["parent_id"] = parent_ids

df_assistant = df_assistant[
    ["instruction", "output", "message_id", "parent_id", "lang", "rank"]
].rename(columns={"message_id": "id"})


# 翻訳タスクのみデータに異常があるので除外
df_assistant2 = df_assistant[~df_assistant["instruction"].str.contains("翻訳")]

In [9]:
# これ以下でjsonファイルへ書き出し---------------

learn_datas = []
input_list = []

for n in range(len(df_assistant2)):
    learn_data = {
        "id": str(df_assistant2.iloc[n, 2]),  # "id"カラムの追加
        "instruction": str(df_assistant2.iloc[n, 0]),
        "input": "",
        "output": ""
    }

    input_list.append(df_assistant2.iloc[n, 0])
    learn_data["input"] = ""
    learn_data["output"] = str(df_assistant2.iloc[n, 1])

    learn_datas.append(learn_data)

json_learn_data = json.dumps(learn_datas, indent=4, ensure_ascii=False)
with open('./data/oasst1-89k-ja.json', 'w', encoding="utf-8") as f:
    f.write(json_learn_data)

## Alpacaのデータセットの形式変更

In [1]:
import json

# JSONLファイルを読み込む
with open('./data/seed_tasks_jp_cleaned.jsonl', 'r', encoding='utf-8') as file:
    data = [json.loads(line) for line in file]

# フォーマットを変更
updated_data = []
for item in data:
    if 'instances' in item:
        for instance in item['instances']:
            new_item = {
                'id': item['id'],
                'name': item['name'],
                'instruction': item['instruction'],
                'input': instance['input'],
                'output': instance['output'],
                'is_classification': item['is_classification']
            }
            updated_data.append(new_item)
    else:
        updated_data.append(item)

# 更新されたデータをJSONLファイルに書き込む
with open('./data/alpaca_seed_tasks_jp.jsonl', 'w', encoding='utf-8') as file:
    for item in updated_data:
        json.dump(item, file, ensure_ascii=False)
        file.write('\n')


## hh-rlhf-49k-jaデータセット

In [12]:
from datasets import Dataset, load_dataset

dataset = load_dataset("kunishou/hh-rlhf-49k-ja")
dataset.set_format(type="pandas")
df = dataset["train"][:]
df = df[df["ng_translation"]!="1"].drop(["ng_translation", "index"], axis=1).reset_index()
df = df.rename(columns={"index": "id"})
df = df[["id", "instruction", "input", "output"]]

# DataFrameをJSONファイルとして保存（日本語を正しく扱うために ensure_ascii=False を設定）
df.to_json("./data/hh-rlhf-49k-ja.jsonl", force_ascii=False, orient="records", lines=True)

In [13]:
from transformers import AutoTokenizer
MODEL_PATH = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

/Users/yagiharahajime/Documents/06_プロジェクト/29_wizardLM/WizardLM/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [23]:
# a="</llm-code>"
a="\\n</llm-code>"
tok=tokenizer.encode(a, add_special_tokens=False)
back=tokenizer.decode([1867, 584, 28719, 28733, 1409, 28767])
a,tok,back

('\\n</llm-code>',
 [414, 28711, 700, 584, 28719, 28733, 1409, 28767],
 '</llm-code>')